In [170]:
import pandas as pd
import pymongo
import pprint
from collections import Counter
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [2]:
from pymongo import MongoClient
client = MongoClient()

In [3]:
#db = client.hkust
db = client.msbd500103

### Load Training Data

In [4]:
trData = db.training_record
#trData = db.training_record

In [5]:
trData = trData.find({})
trData = list(trData)
df_trData = pd.DataFrame(trData)
df_trData.head()

,_id,article_contentid,article_dwelltime,article_l1_categories,article_l2_categories,article_lan,article_publisher,article_title,article_update_time,dt,...,user_country,user_gender,user_gender_weight,user_gp_frequency,user_gp_keywords,user_id,user_maxmind_city,user_maxmind_country_iso_code,user_maxmind_state_iso_code,user_model
0,5a07224d6c0c65269105e6c7,1l5c17e72og_us,NaN,NaN,NaN,en,NaN,NaN,NaN,20170111,...,US,2.0,0.763295,"{u'SHOPPING': 1, u'BUSINESS': 1, u'ENTERTAINME...",NaN,614E304DE3B07EBCDD4E11382578EF5A,Glen Allen,US,VA,SPH-L710
1,5a07224d6c0c65269105e6c8,mQ56173a2KT_us,NaN,NaN,NaN,en,NaN,NaN,NaN,20170117,...,US,2.0,0.763295,"{u'SHOPPING': 1, u'BUSINESS': 1, u'ENTERTAINME...",NaN,614E304DE3B07EBCDD4E11382578EF5A,Glen Allen,US,VA,SPH-L710
2,5a07224d6c0c65269105e6c9,Ly7407d4cuC_us,31.0,"[{u'name': u'1000780', u'weight': 0.4546442536...","[{u'name': u'1000667', u'weight': 0.6867789988...",en,The Sun,"Dad caught selling his baby son for £9,000 to ...",1.484656e+09,20170117,...,US,2.0,0.763295,"{u'SHOPPING': 1, u'BUSINESS': 1, u'ENTERTAINME...",NaN,614E304DE3B07EBCDD4E11382578EF5A,Glen Allen,US,VA,SPH-L710
3,5a07224d6c0c65269105e6ca,AY628126a1w_us,NaN,"[{u'name': u'1000288', u'weight': 0.9748942874...","[{u'name': u'1000288', u'weight': 0.44672383556}]",en,Bustle,11 Weird Ways Your Relationship With Your Pare...,1.482379e+09,20170110,...,US,2.0,0.763295,"{u'SHOPPING': 1, u'BUSINESS': 1, u'ENTERTAINME...",NaN,614E304DE3B07EBCDD4E11382578EF5A,Glen Allen,US,VA,SPH-L710
4,5a07224d6c0c65269105e6cb,jv70536a3bt_us,NaN,"[{u'name': u'1000031', u'weight': 0.4396004700...","[{u'name': u'1000069', u'weight': 0.8568055895...",en,Los Angeles Times,Ivanka Trump goes public about exiting fashion...,1.484223e+09,20170113,...,US,2.0,0.763295,"{u'SHOPPING': 1, u'BUSINESS': 1, u'ENTERTAINME...",NaN,614E304DE3B07EBCDD4E11382578EF5A,Glen Allen,US,VA,SPH-L710


In [6]:
df_trData = df_trData[~df_trData['user_gp_frequency'].isnull()]
df_trData = df_trData[~df_trData['article_title'].isnull()]

In [7]:
#sophia remark get the mapping of user article
df_ua_map = df_trData[['user_id', 'article_contentid']]
df_ua_map.shape

(168276, 2)

In [8]:
df_ua_map = df_ua_map.drop_duplicates(subset = ['user_id', 'article_contentid'], keep='first')
df_ua_map.shape

(139811, 2)

In [9]:
#df_ua_map.to_csv('data1202/df_ua_map.csv')

### Load Test User

In [10]:
tstData = db.test_user
tstData = tstData.find({})
tstData = list(tstData)
df_tstData = pd.DataFrame(tstData)
df_tstData.head()

,_id,user_age,user_all_pkgs,user_brand,user_country,user_gender,user_gender_weight,user_gp_frequency,user_gp_keywords,user_id,user_maxmind_city,user_maxmind_country_iso_code,user_maxmind_state_iso_code,user_model
0,5a0722a76c0c65269108f616,3.0,"[com.pinterest, com.cvs.launchers.cvs, com.dom...",samsung,US,2.0,0.852654,"{u'PRODUCTIVITY': 1, u'COMMUNICATION': 1, u'TR...",NaN,CD2F2B61135DB27BA3230F91466ED793,Lebanon,US,PA,SAMSUNG-SM-G890A
1,5a0722a76c0c65269108f617,2.0,"[com.ebay.mobile, com.mobilesrepublic.appy, de...",samsung,US,1.0,0.355348,"{u'HOUSE_AND_HOME': 1, u'SHOPPING': 1, u'FINAN...",NaN,5BA72C11B1E94BA50F1E66F4B1362CE6,Fort Wayne,US,IN,SM-T217S
2,5a0722a76c0c65269108f618,4.0,"[com.t11.skyviewfree, com.zendesk.android, com...",motorola,US,2.0,0.810239,"{u'GAME_CARD': 5, u'PRODUCTIVITY': 5, u'FINANC...",NaN,40BE601804C6B2E842FFC30F0EDBAC23,Tualatin,US,OR,XT1254
3,5a0722a76c0c65269108f619,4.0,"[com.t11.skyviewfree, com.zendesk.android, com...",motorola,US,2.0,0.810239,"{u'GAME_CARD': 5, u'PRODUCTIVITY': 5, u'FINANC...",NaN,40BE601804C6B2E842FFC30F0EDBAC23,Tualatin,US,OR,XT1254
4,5a0722a76c0c65269108f61a,2.0,"[com.wetv.WEtviPhoneApp, com.gomo.battery, com...",zte,US,2.0,0.891180,"{u'VIDEO_PLAYERS': 1, u'PRODUCTIVITY': 4, u'GA...",NaN,E1295F6622EA8DA36DE678772D9ED152,Riverview,US,FL,Z787


In [11]:
#df_tstData.drop_duplicates(['user_id'])['user_id'].to_csv('data1202/df_tst_user.csv', sep='\t', encoding='utf-8')
#df_tstData.drop_duplicates(['user_id'])[['user_id', 'user_age','user_gender']].to_csv('data1202/df_tst_user_d.csv', sep='\t', encoding='utf-8')

In [169]:
df_tstData.drop_duplicates(['user_id']).shape

(2970, 14)

### Prepare User View: df_tr_user vs df_tst_user¶

In [12]:
df_tr_user = df_trData[['user_id', 'user_age', 'user_gender', 'user_brand', 'user_model','user_maxmind_city']]
df_tst_user = df_tstData[['user_id', 'user_age', 'user_gender', 'user_brand', 'user_model', 'user_maxmind_city']]
df_tr_user.head()

,user_id,user_age,user_gender,user_brand,user_model,user_maxmind_city
2,614E304DE3B07EBCDD4E11382578EF5A,2.0,2.0,samsung,SPH-L710,Glen Allen
3,614E304DE3B07EBCDD4E11382578EF5A,2.0,2.0,samsung,SPH-L710,Glen Allen
4,614E304DE3B07EBCDD4E11382578EF5A,2.0,2.0,samsung,SPH-L710,Glen Allen
5,614E304DE3B07EBCDD4E11382578EF5A,2.0,2.0,samsung,SPH-L710,Glen Allen
6,CD2F2B61135DB27BA3230F91466ED793,3.0,2.0,samsung,SAMSUNG-SM-G890A,Lebanon


In [13]:
df_tst_user.head()

,user_id,user_age,user_gender,user_brand,user_model,user_maxmind_city
0,CD2F2B61135DB27BA3230F91466ED793,3.0,2.0,samsung,SAMSUNG-SM-G890A,Lebanon
1,5BA72C11B1E94BA50F1E66F4B1362CE6,2.0,1.0,samsung,SM-T217S,Fort Wayne
2,40BE601804C6B2E842FFC30F0EDBAC23,4.0,2.0,motorola,XT1254,Tualatin
3,40BE601804C6B2E842FFC30F0EDBAC23,4.0,2.0,motorola,XT1254,Tualatin
4,E1295F6622EA8DA36DE678772D9ED152,2.0,2.0,zte,Z787,Riverview


In [14]:
df_user = pd.concat([df_tr_user, df_tst_user])

In [15]:
df_user['user_age'] = df_user['user_age'].fillna(0)
df_user['user_gender'] = df_user['user_gender'].fillna(0)
df_user['user_brand'] = df_user['user_brand'].str.lower().fillna('na')
df_user['user_maxmind_city'] = df_user['user_maxmind_city'].str.lower().fillna('na')
df_user['user_model'] = df_user['user_model'].str.lower().fillna('na')
df_user.head()

,user_id,user_age,user_gender,user_brand,user_model,user_maxmind_city
2,614E304DE3B07EBCDD4E11382578EF5A,2.0,2.0,samsung,sph-l710,glen allen
3,614E304DE3B07EBCDD4E11382578EF5A,2.0,2.0,samsung,sph-l710,glen allen
4,614E304DE3B07EBCDD4E11382578EF5A,2.0,2.0,samsung,sph-l710,glen allen
5,614E304DE3B07EBCDD4E11382578EF5A,2.0,2.0,samsung,sph-l710,glen allen
6,CD2F2B61135DB27BA3230F91466ED793,3.0,2.0,samsung,samsung-sm-g890a,lebanon


In [16]:
df_user = pd.get_dummies(df_user, columns=['user_age', 'user_gender','user_brand', 'user_maxmind_city', 'user_model'])
df_user = df_user.fillna(0)
df_user = df_user.drop_duplicates(subset=['user_id'], keep='first')
df_user.head()

,user_id,user_age_0.0,user_age_1.0,user_age_2.0,user_age_3.0,user_age_4.0,user_gender_0.0,user_gender_1.0,user_gender_2.0,user_brand_acer,...,user_model_z955l,user_model_z958,user_model_z959,user_model_z963u,user_model_z963vl,user_model_z970,user_model_z981,user_model_z987,user_model_z988,user_model_zte_n9511
2,614E304DE3B07EBCDD4E11382578EF5A,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6,CD2F2B61135DB27BA3230F91466ED793,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
11,56F27AD705D8564F7900470AA793DBD2,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
12,8EE461BA3AA2595AC91FCDD833EC1711,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
15,5BA72C11B1E94BA50F1E66F4B1362CE6,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
train_user_id = df_trData.drop_duplicates(subset=['user_id'], keep='first')['user_id'].values
test_user_id = df_tstData.drop_duplicates(subset=['user_id'], keep='first')['user_id'].values

In [18]:
df_user.shape

(8044, 3040)

In [19]:
df_tr_user = df_user[df_user['user_id'].isin(train_user_id)]
df_tr_user.shape

(7857, 3040)

In [20]:
df_tst_user = df_user[df_user['user_id'].isin(test_user_id)]
df_tst_user.shape

(2970, 3040)

### Prepare App View

In [21]:
df_tr_app = df_trData[['user_id', 'user_gp_frequency']]
df_tr_app.head()

,user_id,user_gp_frequency
2,614E304DE3B07EBCDD4E11382578EF5A,"{u'SHOPPING': 1, u'BUSINESS': 1, u'ENTERTAINME..."
3,614E304DE3B07EBCDD4E11382578EF5A,"{u'SHOPPING': 1, u'BUSINESS': 1, u'ENTERTAINME..."
4,614E304DE3B07EBCDD4E11382578EF5A,"{u'SHOPPING': 1, u'BUSINESS': 1, u'ENTERTAINME..."
5,614E304DE3B07EBCDD4E11382578EF5A,"{u'SHOPPING': 1, u'BUSINESS': 1, u'ENTERTAINME..."
6,CD2F2B61135DB27BA3230F91466ED793,"{u'PRODUCTIVITY': 1, u'COMMUNICATION': 1, u'TR..."


In [22]:
df_tst_app = df_tstData[['user_id', 'user_gp_frequency']]

In [23]:
df_tst_na_app = df_tst_app[df_tst_app['user_gp_frequency'].isnull()]
df_tst_na_app = df_tst_na_app.drop(['user_gp_frequency'],axis = 1)
df_tst_na_app['NOAPP'] = 1
df_tst_na_app.shape

(89, 2)

In [24]:
df_app = pd.concat([df_tr_app, df_tst_app])
df_app = df_app.dropna()

In [25]:
df_app_category = pd.DataFrame.from_records(df_app['user_gp_frequency'].tolist())
df_app_category = df_app_category.fillna(0)
df_app_category['NOAPP'] = 0
df_app_category.head()

,ART_AND_DESIGN,AUTO_AND_VEHICLES,BEAUTY,BOOKS_AND_REFERENCE,BUSINESS,COMICS,COMMUNICATION,DATING,EDUCATION,ENTERTAINMENT,...,PRODUCTIVITY,SHOPPING,SOCIAL,SPORTS,TOOLS,TRANSPORTATION,TRAVEL_AND_LOCAL,VIDEO_PLAYERS,WEATHER,NOAPP
0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,2.0,...,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0,0
1,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,2.0,...,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0,0
2,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,2.0,...,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0,0
3,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,2.0,...,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,3.0,0.0,1.0,0.0,0.0,0


In [26]:
df_app = df_app.merge(df_app_category, how='left', left_index=True, right_index=True)
df_app = df_app.drop(['user_gp_frequency'], axis=1)
df_app = df_app.drop_duplicates(subset=['user_id'], keep='first')
df_app.head()

,user_id,ART_AND_DESIGN,AUTO_AND_VEHICLES,BEAUTY,BOOKS_AND_REFERENCE,BUSINESS,COMICS,COMMUNICATION,DATING,EDUCATION,...,PRODUCTIVITY,SHOPPING,SOCIAL,SPORTS,TOOLS,TRANSPORTATION,TRAVEL_AND_LOCAL,VIDEO_PLAYERS,WEATHER,NOAPP
0,CD2F2B61135DB27BA3230F91466ED793,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0,0
1,5BA72C11B1E94BA50F1E66F4B1362CE6,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0,0
2,614E304DE3B07EBCDD4E11382578EF5A,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0,0
2,40BE601804C6B2E842FFC30F0EDBAC23,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0,0
4,E1295F6622EA8DA36DE678772D9ED152,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,3.0,0.0,1.0,0.0,0.0,0


In [27]:
df_app = pd.concat([df_app, df_tst_na_app])
df_app.head()


,ART_AND_DESIGN,AUTO_AND_VEHICLES,BEAUTY,BOOKS_AND_REFERENCE,BUSINESS,COMICS,COMMUNICATION,DATING,EDUCATION,ENTERTAINMENT,...,PRODUCTIVITY,SHOPPING,SOCIAL,SPORTS,TOOLS,TRANSPORTATION,TRAVEL_AND_LOCAL,VIDEO_PLAYERS,WEATHER,user_id
0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,2.0,...,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0,CD2F2B61135DB27BA3230F91466ED793
1,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,2.0,...,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0,5BA72C11B1E94BA50F1E66F4B1362CE6
2,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,2.0,...,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0,614E304DE3B07EBCDD4E11382578EF5A
2,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,2.0,...,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0,40BE601804C6B2E842FFC30F0EDBAC23
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,3.0,0.0,1.0,0.0,0.0,E1295F6622EA8DA36DE678772D9ED152


In [28]:
def sumCat(row):
    l = row.tolist()
    #print l
    row['totalCat'] = sum(l[1:])
    return row

In [29]:
df_app1 = df_app.drop(['user_id'], axis = 1).apply(lambda row: sumCat(row), axis = 1)
df_app1 = df_app[['user_id']].merge(df_app1, how = 'left', left_index=True, right_index=True)
df_app2 = df_app1[df_app1['totalCat'] != 0].drop(['totalCat'], axis = 1)

In [30]:
df_app2.shape

(8309, 54)

In [31]:
df_app2 = df_app2.drop_duplicates(subset=['user_id'], keep='first')

In [32]:
df_app2.shape

(8044, 54)

In [33]:
df_app2.head()

,user_id,ART_AND_DESIGN,AUTO_AND_VEHICLES,BEAUTY,BOOKS_AND_REFERENCE,BUSINESS,COMICS,COMMUNICATION,DATING,EDUCATION,...,PHOTOGRAPHY,PRODUCTIVITY,SHOPPING,SOCIAL,SPORTS,TOOLS,TRANSPORTATION,TRAVEL_AND_LOCAL,VIDEO_PLAYERS,WEATHER
0,CD2F2B61135DB27BA3230F91466ED793,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0
1,5BA72C11B1E94BA50F1E66F4B1362CE6,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0
2,614E304DE3B07EBCDD4E11382578EF5A,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0
2,40BE601804C6B2E842FFC30F0EDBAC23,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0
4,E1295F6622EA8DA36DE678772D9ED152,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,2.0,1.0,0.0,1.0,0.0,3.0,0.0,1.0,0.0,0.0


In [34]:
app_category = df_app.drop(['user_id'], axis = 1).columns
app_category

Index([     u'ART_AND_DESIGN',   u'AUTO_AND_VEHICLES',              u'BEAUTY',
       u'BOOKS_AND_REFERENCE',            u'BUSINESS',              u'COMICS',
             u'COMMUNICATION',              u'DATING',           u'EDUCATION',
             u'ENTERTAINMENT',              u'EVENTS',       u'FAMILY_ACTION',
                   u'FINANCE',      u'FOOD_AND_DRINK',         u'GAME_ACTION',
            u'GAME_ADVENTURE',         u'GAME_ARCADE',          u'GAME_BOARD',
                 u'GAME_CARD',         u'GAME_CASINO',         u'GAME_CASUAL',
          u'GAME_EDUCATIONAL',          u'GAME_MUSIC',         u'GAME_PUZZLE',
               u'GAME_RACING',   u'GAME_ROLE_PLAYING',     u'GAME_SIMULATION',
               u'GAME_SPORTS',       u'GAME_STRATEGY',         u'GAME_TRIVIA',
                 u'GAME_WORD',  u'HEALTH_AND_FITNESS',      u'HOUSE_AND_HOME',
        u'LIBRARIES_AND_DEMO',           u'LIFESTYLE', u'MAPS_AND_NAVIGATION',
           u'MEDIA_AND_VIDEO',             u'MEDICAL

## Merge user and app

In [35]:
df_tr_user.shape, df_app2.shape

((7857, 3040), (8044, 54))

In [36]:
df_tr_user_app = df_tr_user.merge(df_app2, how = 'left', on = 'user_id')
df_tr_user_app = df_tr_user_app.fillna(0) #change from fillna(0) to fillna(0.1)
df_tr_user_app.head()

,user_id,user_age_0.0,user_age_1.0,user_age_2.0,user_age_3.0,user_age_4.0,user_gender_0.0,user_gender_1.0,user_gender_2.0,user_brand_acer,...,PHOTOGRAPHY,PRODUCTIVITY,SHOPPING,SOCIAL,SPORTS,TOOLS,TRANSPORTATION,TRAVEL_AND_LOCAL,VIDEO_PLAYERS,WEATHER
0,614E304DE3B07EBCDD4E11382578EF5A,0,0,1,0,0,0,0,1,0,...,0.0,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0
1,CD2F2B61135DB27BA3230F91466ED793,0,0,0,1,0,0,0,1,0,...,0.0,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0
2,56F27AD705D8564F7900470AA793DBD2,0,0,0,0,1,0,0,1,0,...,2.0,1.0,2.0,3.0,1.0,5.0,0.0,1.0,2.0,0.0
3,8EE461BA3AA2595AC91FCDD833EC1711,0,0,1,0,0,0,0,1,0,...,2.0,1.0,2.0,3.0,1.0,5.0,0.0,1.0,2.0,0.0
4,5BA72C11B1E94BA50F1E66F4B1362CE6,0,0,1,0,0,0,1,0,0,...,0.0,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0


In [37]:
df_tst_user_app = df_tst_user.merge(df_app2, how='left',on='user_id')
df_tr_user_app = df_tr_user_app.fillna(0)
df_tr_user_app.head()

,user_id,user_age_0.0,user_age_1.0,user_age_2.0,user_age_3.0,user_age_4.0,user_gender_0.0,user_gender_1.0,user_gender_2.0,user_brand_acer,...,PHOTOGRAPHY,PRODUCTIVITY,SHOPPING,SOCIAL,SPORTS,TOOLS,TRANSPORTATION,TRAVEL_AND_LOCAL,VIDEO_PLAYERS,WEATHER
0,614E304DE3B07EBCDD4E11382578EF5A,0,0,1,0,0,0,0,1,0,...,0.0,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0
1,CD2F2B61135DB27BA3230F91466ED793,0,0,0,1,0,0,0,1,0,...,0.0,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0
2,56F27AD705D8564F7900470AA793DBD2,0,0,0,0,1,0,0,1,0,...,2.0,1.0,2.0,3.0,1.0,5.0,0.0,1.0,2.0,0.0
3,8EE461BA3AA2595AC91FCDD833EC1711,0,0,1,0,0,0,0,1,0,...,2.0,1.0,2.0,3.0,1.0,5.0,0.0,1.0,2.0,0.0
4,5BA72C11B1E94BA50F1E66F4B1362CE6,0,0,1,0,0,0,1,0,0,...,0.0,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0


### Load Test Article

In [52]:
tstArt = db.test_article
tstArt = tstArt.find({})
tstArt = list(tstArt)
df_tstArt = pd.DataFrame(tstArt)
df_tstArt.head()

,_id,article_contentid,article_l1_categories,article_l2_categories,article_lan,article_publisher,article_title,article_update_time
0,5a0722856c0c65269108a0b6,sq7f5162eGP_us,"[{u'name': u'1000560', u'weight': 0.7253321861...","[{u'name': u'1000596', u'weight': 0.9452144284...",en,Spasique,"Nastiest Thing You Are Willing To Do In Bed, B...",1.485686e+09
1,5a0722856c0c65269108a0b7,dh7f0517aP4_us,"[{u'name': u'1000031', u'weight': 0.8983825108...","[{u'name': u'1000069', u'weight': 0.7455885870...",en,Vulture,Matthew McConaughey and Anne Hathaway Are in T...,1.485696e+09
2,5a0722856c0c65269108a0b8,AE78003f4fk_us,"[{u'name': u'1000031', u'weight': 0.9555669128...","[{u'name': u'1000031', u'weight': 0.9555669128...",en,The Hollywood Gossip,Love & Hip Hop Cast Rankings: Who Can Actually...,1.485084e+09
3,5a0722856c0c65269108a0b9,oj7e81b559q_us,"[{u'name': u'1000031', u'weight': 0.9492414325...","[{u'name': u'1000069', u'weight': 0.7962460693...",en,LBCC Viking News,Rasheeda From 'Love and Hip Hop Atlanta' Respo...,1.485685e+09
4,5a0722856c0c65269108a0ba,h57baa257GX_us,"[{u'name': u'1000931', u'weight': 0.5076023340...","[{u'name': u'1000931', u'weight': 0.5076023340...",en,Smithsonian Channel,Brave Diver Spots a GIANT Fish. But When The C...,1.485714e+09


In [53]:
#df_tstArt.drop_duplicates(['article_contentid'])['article_contentid'].to_csv('data1202/df_tst_art.csv', sep = '\t', encoding='utf-8')
df_tstArt.drop_duplicates(['article_contentid'])[['article_contentid','article_publisher','article_update_time']].to_csv('data1202/df_tst_art_1.csv', sep = '\t', encoding='utf-8')

### Prepare Article View

In [54]:
df_tr_article = df_trData[['user_id', 'article_contentid', 'article_l1_categories', 'article_l2_categories', 'article_title', 'dt']]#sophia remark : should add article_update_time as a feature
df_tr_article = df_tr_article.sort_values(by=['article_title'], ascending=False)
df_tr_article.head()

,user_id,article_contentid,article_l1_categories,article_l2_categories,article_title,dt
73696,7C62CF4B33EF7303881DB654E68FF85E,v77c577e1qo_us,"[{u'name': u'1000780', u'weight': 0.8786688469...","[{u'name': u'1000667', u'weight': 0.4648142188...",﻿Vulnerable teen 'livestreams suicide' on Face...,20170126
125132,A4F138114721E738AD8C2768C11450A9,v77c577e1qo_us,"[{u'name': u'1000780', u'weight': 0.8786688469...","[{u'name': u'1000667', u'weight': 0.4648142188...",﻿Vulnerable teen 'livestreams suicide' on Face...,20170125
45894,986759E2FD77D818BEDC17BFF6376900,v77c577e1qo_us,"[{u'name': u'1000780', u'weight': 0.8786688469...","[{u'name': u'1000667', u'weight': 0.4648142188...",﻿Vulnerable teen 'livestreams suicide' on Face...,20170126
136028,17A177FDE5C3C9F500D784FF8D985AD7,0i65cb6eaNb_us,"[{u'name': u'1000780', u'weight': 0.0405336841...","[{u'name': u'1000780', u'weight': 0.0405336841...",【E-girls】ちび丸子ちゃん―踊りを学びましょう～,20170112
136080,17A177FDE5C3C9F500D784FF8D985AD7,0i65cb6eaNb_us,"[{u'name': u'1000780', u'weight': 0.0405336841...","[{u'name': u'1000780', u'weight': 0.0405336841...",【E-girls】ちび丸子ちゃん―踊りを学びましょう～,20170112


In [55]:
from sklearn import preprocessing
#Create x, where x the 'scores' column's values as floats
y = df_tr_article['dt'].values.astype(float)
y = y.reshape(-1,1)
# Create a minimum and maximum processor object
min_max_scaler = preprocessing.MinMaxScaler()

# Create an object to transform the data to fit minmax processor
y_scaled = min_max_scaler.fit_transform(y)
y_scaled
# Run the normalizer on the dataframe
df_tr_article['dt'] = y_scaled

In [56]:
df_tst_article = df_tstArt[['article_contentid', 'article_l1_categories', 'article_l2_categories', 'article_title']]#sophia remark : should add article_update_time as a feature
df_tst_article = df_tst_article.sort_values(by=['article_title'], ascending=False)
df_tst_article = df_tst_article.drop_duplicates(['article_contentid'])
df_tst_article.head()

,article_contentid,article_l1_categories,article_l2_categories,article_title
14013,AI7da9ea5ts_us,"[{u'name': u'1000395', u'weight': 0.2770316004...","[{u'name': u'1000395', u'weight': 0.2770316004...",❀ This HAPPENS If You DIP YOUR FEET IN VINEGAR...
16519,8v7f3eb216n_us,"[{u'name': u'1000111', u'weight': 0.9228855371...","[{u'name': u'1000111', u'weight': 0.9228855371...",★ HERE'S the Misplaced Hair Tutorial I Didn't ...
437,367e61137BH_us,"[{u'name': u'1000395', u'weight': 0.6482318817...","[{u'name': u'1000495', u'weight': 0.6262967681...","‘When my son was born, we altered his body for..."
7018,Kx81111cfRK_us,"[{u'name': u'1000780', u'weight': 0.3613779566...","[{u'name': u'1000780', u'weight': 0.5746450411...",‘We have a new friend’: Teenager’s tweet from ...
7534,Qx784e225N3_us,"[{u'name': u'1000780', u'weight': 0.8869684420...","[{u'name': u'1000804', u'weight': 0.8519911868...",‘We Are Living History’: 4 Famous Pastors Have...


In [57]:
df_tst_article.shape

(8258, 4)

In [58]:
df_article_temp = df_tr_article[['article_contentid', 'dt']].drop_duplicates(['article_contentid'])

df_tst_article = df_tst_article.merge(df_article_temp[['article_contentid', 'dt']], how = 'left', on = 'article_contentid')
df_tst_article.head()

,article_contentid,article_l1_categories,article_l2_categories,article_title,dt
0,AI7da9ea5ts_us,"[{u'name': u'1000395', u'weight': 0.2770316004...","[{u'name': u'1000395', u'weight': 0.2770316004...",❀ This HAPPENS If You DIP YOUR FEET IN VINEGAR...,NaN
1,8v7f3eb216n_us,"[{u'name': u'1000111', u'weight': 0.9228855371...","[{u'name': u'1000111', u'weight': 0.9228855371...",★ HERE'S the Misplaced Hair Tutorial I Didn't ...,NaN
2,367e61137BH_us,"[{u'name': u'1000395', u'weight': 0.6482318817...","[{u'name': u'1000495', u'weight': 0.6262967681...","‘When my son was born, we altered his body for...",1.000000
3,Kx81111cfRK_us,"[{u'name': u'1000780', u'weight': 0.3613779566...","[{u'name': u'1000780', u'weight': 0.5746450411...",‘We have a new friend’: Teenager’s tweet from ...,NaN
4,Qx784e225N3_us,"[{u'name': u'1000780', u'weight': 0.8869684420...","[{u'name': u'1000804', u'weight': 0.8519911868...",‘We Are Living History’: 4 Famous Pastors Have...,0.666667


In [59]:
df_tst_article['dt'] = df_tst_article['dt'].fillna(1)
df_tst_article.head()

,article_contentid,article_l1_categories,article_l2_categories,article_title,dt
0,AI7da9ea5ts_us,"[{u'name': u'1000395', u'weight': 0.2770316004...","[{u'name': u'1000395', u'weight': 0.2770316004...",❀ This HAPPENS If You DIP YOUR FEET IN VINEGAR...,1.000000
1,8v7f3eb216n_us,"[{u'name': u'1000111', u'weight': 0.9228855371...","[{u'name': u'1000111', u'weight': 0.9228855371...",★ HERE'S the Misplaced Hair Tutorial I Didn't ...,1.000000
2,367e61137BH_us,"[{u'name': u'1000395', u'weight': 0.6482318817...","[{u'name': u'1000495', u'weight': 0.6262967681...","‘When my son was born, we altered his body for...",1.000000
3,Kx81111cfRK_us,"[{u'name': u'1000780', u'weight': 0.3613779566...","[{u'name': u'1000780', u'weight': 0.5746450411...",‘We have a new friend’: Teenager’s tweet from ...,1.000000
4,Qx784e225N3_us,"[{u'name': u'1000780', u'weight': 0.8869684420...","[{u'name': u'1000804', u'weight': 0.8519911868...",‘We Are Living History’: 4 Famous Pastors Have...,0.666667


In [60]:
df_tst_article.shape

(8258, 5)

In [61]:
df_tst_article['user_id'] = 'na'

In [62]:
#df_tst_article.head()

In [63]:
tst_article_list = df_tst_article['article_contentid'].values

In [64]:
df_tr_article = df_tr_article.drop_duplicates(subset=['article_contentid'], keep='first')
df_tr_article = df_tr_article.reset_index(drop = True)
df_tr_article.head()

,user_id,article_contentid,article_l1_categories,article_l2_categories,article_title,dt
0,7C62CF4B33EF7303881DB654E68FF85E,v77c577e1qo_us,"[{u'name': u'1000780', u'weight': 0.8786688469...","[{u'name': u'1000667', u'weight': 0.4648142188...",﻿Vulnerable teen 'livestreams suicide' on Face...,0.888889
1,17A177FDE5C3C9F500D784FF8D985AD7,0i65cb6eaNb_us,"[{u'name': u'1000780', u'weight': 0.0405336841...","[{u'name': u'1000780', u'weight': 0.0405336841...",【E-girls】ちび丸子ちゃん―踊りを学びましょう～,0.111111
2,DB7995E79922E8A96E1585DA8C6EFB51,ag639e0eeDK_us,"[{u'name': u'8000001', u'weight': 0.999}]","[{u'name': u'8000001', u'weight': 0.999}]",♠ Funny Videos ♠ Best FAILS ♠ December 2016 Co...,0.055556
3,84E7779E484630375F5F0F6175045B4B,7m799a2c4mX_us,"[{u'name': u'1000931', u'weight': 0.1268057078...","[{u'name': u'1000931', u'weight': 0.1268057078...",▶[Future Bass] ★ MRVLZ - On The Inside,0.833333
4,DE83F55126D386CF852A76F25FE0FE04,hQ7cfd994Tv_us,"[{u'name': u'1000780', u'weight': 0.4859073658...","[{u'name': u'1000780', u'weight': 0.4859073658...",€1m in funding sought for new air ambulance se...,0.944444


In [65]:
df_tr_article.shape

(49149, 6)

In [66]:
df_article = pd.concat([df_tr_article, df_tst_article])
df_article.head()

,article_contentid,article_l1_categories,article_l2_categories,article_title,dt,user_id
0,v77c577e1qo_us,"[{u'name': u'1000780', u'weight': 0.8786688469...","[{u'name': u'1000667', u'weight': 0.4648142188...",﻿Vulnerable teen 'livestreams suicide' on Face...,0.888889,7C62CF4B33EF7303881DB654E68FF85E
1,0i65cb6eaNb_us,"[{u'name': u'1000780', u'weight': 0.0405336841...","[{u'name': u'1000780', u'weight': 0.0405336841...",【E-girls】ちび丸子ちゃん―踊りを学びましょう～,0.111111,17A177FDE5C3C9F500D784FF8D985AD7
2,ag639e0eeDK_us,"[{u'name': u'8000001', u'weight': 0.999}]","[{u'name': u'8000001', u'weight': 0.999}]",♠ Funny Videos ♠ Best FAILS ♠ December 2016 Co...,0.055556,DB7995E79922E8A96E1585DA8C6EFB51
3,7m799a2c4mX_us,"[{u'name': u'1000931', u'weight': 0.1268057078...","[{u'name': u'1000931', u'weight': 0.1268057078...",▶[Future Bass] ★ MRVLZ - On The Inside,0.833333,84E7779E484630375F5F0F6175045B4B
4,hQ7cfd994Tv_us,"[{u'name': u'1000780', u'weight': 0.4859073658...","[{u'name': u'1000780', u'weight': 0.4859073658...",€1m in funding sought for new air ambulance se...,0.944444,DE83F55126D386CF852A76F25FE0FE04


In [67]:
tr_article_list = df_tr_article['article_contentid'].values

In [68]:
def getDetail(cat, field):
    if (str(cat) != 'nan'):
        if (len(cat) > 0):
            return cat[0][field]
    else:
        return '0'

In [69]:
df_article['l1_cat'] = df_article.apply(lambda row: getDetail(row['article_l1_categories'],'name'), axis=1)
df_article['l2_cat'] = df_article.apply(lambda row: getDetail(row['article_l2_categories'],'name'), axis=1)
df_article['l1_cat_w'] = df_article.apply(lambda row: getDetail(row['article_l1_categories'],'weight'), axis=1)
df_article['l2_cat_w'] = df_article.apply(lambda row: getDetail(row['article_l2_categories'],'weight'), axis=1)
df_article = df_article.drop(['article_l1_categories', 'article_l2_categories'], axis=1)
df_article.head()

,article_contentid,article_title,dt,user_id,l1_cat,l2_cat,l1_cat_w,l2_cat_w
0,v77c577e1qo_us,﻿Vulnerable teen 'livestreams suicide' on Face...,0.888889,7C62CF4B33EF7303881DB654E68FF85E,1000780,1000667,0.878669,0.464814
1,0i65cb6eaNb_us,【E-girls】ちび丸子ちゃん―踊りを学びましょう～,0.111111,17A177FDE5C3C9F500D784FF8D985AD7,1000780,1000780,0.0405337,0.0405337
2,ag639e0eeDK_us,♠ Funny Videos ♠ Best FAILS ♠ December 2016 Co...,0.055556,DB7995E79922E8A96E1585DA8C6EFB51,8000001,8000001,0.999,0.999
3,7m799a2c4mX_us,▶[Future Bass] ★ MRVLZ - On The Inside,0.833333,84E7779E484630375F5F0F6175045B4B,1000931,1000931,0.126806,0.126806
4,hQ7cfd994Tv_us,€1m in funding sought for new air ambulance se...,0.944444,DE83F55126D386CF852A76F25FE0FE04,1000780,1000780,0.485907,0.485907


In [70]:
df_article.shape

(57407, 8)

In [71]:
df_article.drop(['article_title'], axis = 1).to_csv('data1202/df_article.csv')

### Clean title

In [ ]:
#skip

from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk import word_tokenize, pos_tag
from nltk.stem.lancaster import LancasterStemmer
import re


In [ ]:
def hashingTitle(title):
    STEMMER = LancasterStemmer()
    if (type(title) != float) & (type(title) != int):
        title = title.lower()
        title = word_tokenize(title)
        title = " ".join([STEMMER.stem(word.strip('!"#$%&\()*+,-./:;<=>?@[\]^_`{|}~').lower()) for word in title])
        titleList = []
        for w in title:
            if len(re.findall("[a-zA-Z]+", w)) > 0:
                f_w = re.findall("[a-zA-Z]+", w)[0].lower()
                f_w = '#'+f_w+'#'
                f_w_tri = [f_w[i:i+3] for i in range(len(f_w)-1) if (len(f_w[i:i+3]) == 3)]
                titleList.extend(f_w_tri)
        return dict(Counter(titleList))
    else:
        return {}

In [72]:
#df_article = df_article.drop(['article_title','clean_article_title'], axis=1)
#Sophia commend : the following line is to be run if title is not included
df_article = df_article.drop(['article_title'], axis=1)
df_article = df_article.dropna()
df_article.head()

,article_contentid,dt,user_id,l1_cat,l2_cat,l1_cat_w,l2_cat_w
0,v77c577e1qo_us,0.888889,7C62CF4B33EF7303881DB654E68FF85E,1000780,1000667,0.878669,0.464814
1,0i65cb6eaNb_us,0.111111,17A177FDE5C3C9F500D784FF8D985AD7,1000780,1000780,0.0405337,0.0405337
2,ag639e0eeDK_us,0.055556,DB7995E79922E8A96E1585DA8C6EFB51,8000001,8000001,0.999,0.999
3,7m799a2c4mX_us,0.833333,84E7779E484630375F5F0F6175045B4B,1000931,1000931,0.126806,0.126806
4,hQ7cfd994Tv_us,0.944444,DE83F55126D386CF852A76F25FE0FE04,1000780,1000780,0.485907,0.485907


In [73]:
df_cat = df_article[['l1_cat', 'l2_cat', 'l1_cat_w', 'l2_cat_w']]
df_cat = pd.get_dummies(df_cat, columns=['l1_cat', 'l2_cat'])
df_cat.head()

,l1_cat_w,l2_cat_w,l1_cat_0,l1_cat_0010000000,l1_cat_1000001,l1_cat_1000031,l1_cat_1000044,l1_cat_1000111,l1_cat_1000123,l1_cat_1000267,...,l2_cat_1001040,l2_cat_1001049,l2_cat_1001060,l2_cat_1001091,l2_cat_1001103,l2_cat_1001110,l2_cat_1001117,l2_cat_2000000,l2_cat_6000007,l2_cat_8000001
0,0.878669,0.464814,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.0405337,0.0405337,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.999,0.999,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0.126806,0.126806,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.485907,0.485907,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [74]:
df_cat = df_cat.merge(df_article[['l1_cat', 'l2_cat']], how='left', left_index=True, right_index=True)
df_cat.head()

,l1_cat_w,l2_cat_w,l1_cat_0,l1_cat_0010000000,l1_cat_1000001,l1_cat_1000031,l1_cat_1000044,l1_cat_1000111,l1_cat_1000123,l1_cat_1000267,...,l2_cat_1001060,l2_cat_1001091,l2_cat_1001103,l2_cat_1001110,l2_cat_1001117,l2_cat_2000000,l2_cat_6000007,l2_cat_8000001,l1_cat,l2_cat
0,0.878669,0.464814,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1000780,1000667
0,0.878669,0.464814,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1000395,1000395
0,0.277032,0.277032,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1000780,1000667
0,0.277032,0.277032,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1000395,1000395
1,0.0405337,0.0405337,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1000780,1000780


In [75]:
def updateCatWeight(row):
    cat = row['l1_cat']
    row['l1_cat_'+str(cat)] = row['l1_cat_w']
    return row

In [76]:
df_cat = df_cat.apply(lambda row: updateCatWeight(row), axis=1)
df_cat.head()

,l1_cat_w,l2_cat_w,l1_cat_0,l1_cat_0010000000,l1_cat_1000001,l1_cat_1000031,l1_cat_1000044,l1_cat_1000111,l1_cat_1000123,l1_cat_1000267,...,l2_cat_1001060,l2_cat_1001091,l2_cat_1001103,l2_cat_1001110,l2_cat_1001117,l2_cat_2000000,l2_cat_6000007,l2_cat_8000001,l1_cat,l2_cat
0,0.878669,0.464814,0,0.0,0,0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,1000780,1000667
0,0.878669,0.464814,0,0.0,0,0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,1000395,1000395
0,0.277032,0.277032,0,0.0,0,0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,1000780,1000667
0,0.277032,0.277032,0,0.0,0,0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,1000395,1000395
1,0.0405337,0.0405337,0,0.0,0,0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,1000780,1000780


In [77]:
df_cat = df_cat.drop(['l1_cat','l2_cat','l1_cat_w', 'l2_cat_w'], axis=1)
df_cat.head()

,l1_cat_0,l1_cat_0010000000,l1_cat_1000001,l1_cat_1000031,l1_cat_1000044,l1_cat_1000111,l1_cat_1000123,l1_cat_1000267,l1_cat_1000288,l1_cat_1000298,...,l2_cat_1001040,l2_cat_1001049,l2_cat_1001060,l2_cat_1001091,l2_cat_1001103,l2_cat_1001110,l2_cat_1001117,l2_cat_2000000,l2_cat_6000007,l2_cat_8000001
0,0,0.0,0,0,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0.0,0,0,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0.0,0,0,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0.0,0,0,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0.0,0,0,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [78]:
df_article = df_article.merge(df_cat, how='left', left_index=True, right_index=True)
#sophia remark: commend here
#df_article = df_article.drop(['article_publisher','article_title'], axis = 1)
df_article.head()

,article_contentid,dt,user_id,l1_cat,l2_cat,l1_cat_w,l2_cat_w,l1_cat_0,l1_cat_0010000000,l1_cat_1000001,...,l2_cat_1001040,l2_cat_1001049,l2_cat_1001060,l2_cat_1001091,l2_cat_1001103,l2_cat_1001110,l2_cat_1001117,l2_cat_2000000,l2_cat_6000007,l2_cat_8000001
0,v77c577e1qo_us,0.888889,7C62CF4B33EF7303881DB654E68FF85E,1000780,1000667,0.878669,0.464814,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
0,v77c577e1qo_us,0.888889,7C62CF4B33EF7303881DB654E68FF85E,1000780,1000667,0.878669,0.464814,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
0,v77c577e1qo_us,0.888889,7C62CF4B33EF7303881DB654E68FF85E,1000780,1000667,0.878669,0.464814,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
0,v77c577e1qo_us,0.888889,7C62CF4B33EF7303881DB654E68FF85E,1000780,1000667,0.878669,0.464814,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
0,AI7da9ea5ts_us,1.000000,na,1000395,1000395,0.277032,0.277032,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0


In [79]:
df_article_2 = df_article.drop(['l1_cat','l2_cat','l1_cat_w','l2_cat_w'], axis = 1)
df_article_2.shape

(106916, 189)

In [80]:
df_tr_article_2=df_article_2[df_article_2['article_contentid'].isin(tr_article_list)].drop_duplicates(['article_contentid'])
df_tst_article_2=df_article_2[df_article_2['article_contentid'].isin(tst_article_list)].drop_duplicates(['article_contentid'])

In [81]:
print df_tr_article_2.shape
df_tr_article_2 = df_tr_article_2[df_tr_article_2['user_id'] != 'na']
df_tr_article_2.head()

(49129, 189)


,article_contentid,dt,user_id,l1_cat_0,l1_cat_0010000000,l1_cat_1000001,l1_cat_1000031,l1_cat_1000044,l1_cat_1000111,l1_cat_1000123,...,l2_cat_1001040,l2_cat_1001049,l2_cat_1001060,l2_cat_1001091,l2_cat_1001103,l2_cat_1001110,l2_cat_1001117,l2_cat_2000000,l2_cat_6000007,l2_cat_8000001
0,v77c577e1qo_us,0.888889,7C62CF4B33EF7303881DB654E68FF85E,0,0.0,0,0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0i65cb6eaNb_us,0.111111,17A177FDE5C3C9F500D784FF8D985AD7,0,0.0,0,0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ag639e0eeDK_us,0.055556,DB7995E79922E8A96E1585DA8C6EFB51,0,0.0,0,0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,7m799a2c4mX_us,0.833333,84E7779E484630375F5F0F6175045B4B,0,0.0,0,0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,hQ7cfd994Tv_us,0.944444,DE83F55126D386CF852A76F25FE0FE04,0,0.0,0,0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [82]:
print df_tst_article_2.shape
df_tst_article_2 = df_tst_article_2.drop(['user_id'],axis = 1)
df_tst_article_2.head()

(8257, 189)


,article_contentid,dt,l1_cat_0,l1_cat_0010000000,l1_cat_1000001,l1_cat_1000031,l1_cat_1000044,l1_cat_1000111,l1_cat_1000123,l1_cat_1000267,...,l2_cat_1001040,l2_cat_1001049,l2_cat_1001060,l2_cat_1001091,l2_cat_1001103,l2_cat_1001110,l2_cat_1001117,l2_cat_2000000,l2_cat_6000007,l2_cat_8000001
0,AI7da9ea5ts_us,1.000000,0,0.0,0,0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8v7f3eb216n_us,1.000000,0,0.0,0,0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,367e61137BH_us,1.000000,0,0.0,0,0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,Kx81111cfRK_us,1.000000,0,0.0,0,0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Qx784e225N3_us,0.666667,0,0.0,0,0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Change by sophia --> should keep all user-article pair

In [125]:
#df_user_article = df_ua_map.merge(df_user, how='left', on='user_id')
#df_user_article.head()

In [126]:
#df_user_article = df_user_article.merge(df_article.drop(['user_id'],axis = 1), how = 'left', on='article_contentid')
#df_user_article.head()

In [127]:
#df_user_article.shape

In [128]:
#df_user_article = df_user_article.fillna(0)
#df_user_article = df_user_article[(df_user_article['l1_cat'] != 0) & (df_user_article['l2_cat'] != 0)]
#df_user_article.head()

In [129]:
#df_ua_user = df_user_article[df_user.columns]
##df_ua_user = df_ua_user.drop(['user_id'], axis=1)
#df_ua_user = df_ua_user.fillna(0)
#df_ua_user.head()

In [130]:
#df_ua_article = df_user_article[df_article.columns]
#df_ua_article = df_ua_article.drop(['user_id', 'article_contentid'], axis=1)
##df_ua_article = df_ua_article.drop(['article_contentid'], axis=1)
#df_ua_article = df_ua_article.fillna(0)
#df_ua_article.head()

In [131]:
#Sophia remark: save current df_app : user article or others contain ascii - cannot save
#df_user.to_csv('testdata1127/df_user.csv', sep='\t', encoding='utf-8')
#df_ua_article.to_csv('testdata1127/df_ua_article.csv', sep='\t', encoding='utf-8')
#df_article.to_csv('testdata1127/df_article.csv', sep='\t', encoding='utf-8')
#df_app.to_csv('testdata1127/df_app.csv')
#df_app2.to_csv('testdata1127/df_app2.csv')

### Prepare Train Test Set

KNN - in final test, there is no need to get final_train and final_test(previous steps)

In [83]:
df_train_user_0 = df_tr_user_app
df_test_user_0 = df_tst_user_app.fillna(0)
df_tr_user_app.shape, df_tst_user_app.shape

((7857, 3093), (2970, 3093))

In [84]:
a = df_train_user_0.drop(['user_id'], axis = 1).as_matrix()
samples = list(a)
neigh = NearestNeighbors(n_neighbors=1)
neigh.fit(samples) 

#get the nearest user index - use for loop later
print(neigh.kneighbors([samples[15]])) [1][0]

[15]


In [85]:
def findNeighbor(row):
    l = row.values
    row['neibor'] = neigh.kneighbors([l]) [1][0][0]
    return row

In [86]:
df_test_user_1 = df_test_user_0.drop(['user_id'],axis = 1).apply(lambda row: findNeighbor(row), axis = 1)#drop(['user_id'],axis = 1)
df_test_user_1.head()

,user_age_0.0,user_age_1.0,user_age_2.0,user_age_3.0,user_age_4.0,user_gender_0.0,user_gender_1.0,user_gender_2.0,user_brand_acer,user_brand_asus,...,PRODUCTIVITY,SHOPPING,SOCIAL,SPORTS,TOOLS,TRANSPORTATION,TRAVEL_AND_LOCAL,VIDEO_PLAYERS,WEATHER,neibor
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0,1
1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0,4
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0,6
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,3.0,0.0,1.0,0.0,0.0,7
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,3.0,0.0,1.0,0.0,0.0,15


In [87]:
df_user_cat.head()

,user_id,user_l2_cat0010000000,user_l2_cat1000001,user_l2_cat1000002,user_l2_cat1000007,user_l2_cat10000101,user_l2_cat1000012,user_l2_cat10000162,user_l2_cat1000017,user_l2_cat10000177,...,user_l2_cat1001040,user_l2_cat1001049,user_l2_cat1001060,user_l2_cat1001091,user_l2_cat1001103,user_l2_cat1001110,user_l2_cat1001117,user_l2_cat2000000,user_l2_cat6000007,user_l2_cat8000001
0,00108FC5F1F3031ADD536548124A4912,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00310455B7B24D5CDEC892351B8D1D51,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,00381B68ACD6CC83635CDDD9B40F0BAF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,003B2215B070CDB50C0819C72E208F0A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,003B8D414199B9C5EB7CF8C94BCC6787,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [88]:
df_train_user = df_train_user_0.merge(df_user_cat, how = 'left', on = 'user_id')

In [89]:
art_cat_col = [i for i in df_train_user.columns if i not in df_train_user_0.columns]

In [90]:
user_fea_col = [i for i in df_train_user.columns \
                if i not in app_category and 'user_maxmind_city' not in i\
                and 'user_brand' not in i  and 'user_id' != i]
len(user_fea_col)

816

In [91]:
test_col = [i for i in df_train_user.columns \
                if i not in app_category and 'user_maxmind_city' not in i]

In [92]:
df_test_user = df_test_user_1.merge(df_train_user[art_cat_col], how = 'left', left_on='neibor', right_index=True)
df_test_user = df_test_user.drop(['neibor'],axis = 1)
df_test_user.head()

,user_age_0.0,user_age_1.0,user_age_2.0,user_age_3.0,user_age_4.0,user_gender_0.0,user_gender_1.0,user_gender_2.0,user_brand_acer,user_brand_asus,...,user_l2_cat1001040,user_l2_cat1001049,user_l2_cat1001060,user_l2_cat1001091,user_l2_cat1001103,user_l2_cat1001110,user_l2_cat1001117,user_l2_cat2000000,user_l2_cat6000007,user_l2_cat8000001
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [93]:
df_test_user.shape, df_train_user.shape

((2970, 3247), (7857, 3248))

In [94]:
#df_train_user = df_user[df_user['user_id'].isin(final_train_user_id)]
#df_train_user = df_train_user.drop(['user_id'], axis=1)
#df_train_user_2 = df_train_user[user_fea_col].fillna(0)
#df_train_user_arr = df_train_user_2.values
#df_train_user_arr.shape

In [95]:
#Sophia remark: get test user list
#df_test_user0 = df_user[df_user['user_id'].isin(final_test_user_id)]

In [96]:
user_fea_col

['user_age_0.0',
 'user_age_1.0',
 'user_age_2.0',
 'user_age_3.0',
 'user_age_4.0',
 'user_gender_0.0',
 'user_gender_1.0',
 'user_gender_2.0',
 u'user_model_0paj5',
 u'user_model_0pcv1',
 u'user_model_0pja10',
 u'user_model_0pja2',
 u'user_model_0pm92',
 u'user_model_2pq93',
 u'user_model_2ps64',
 u'user_model_2pyb2',
 u'user_model_306sh',
 u'user_model_5017b',
 u'user_model_5027b',
 u'user_model_5042t',
 u'user_model_5054n',
 u'user_model_5054w',
 u'user_model_5056e',
 u'user_model_5056n',
 u'user_model_5065n',
 u'user_model_5098s',
 u'user_model_5560s',
 u'user_model_6045i',
 u'user_model_6045o',
 u'user_model_7040n',
 u'user_model_7040t',
 u'user_model_710c',
 u'user_model_801es',
 u'user_model_831c',
 u'user_model_9006w',
 u'user_model_9020a',
 u'user_model_a0001',
 u'user_model_a1',
 u'user_model_a1-810',
 u'user_model_a1x plus',
 u'user_model_a460g',
 u'user_model_a462c',
 u'user_model_a463bg',
 u'user_model_a464bg',
 u'user_model_a521l',
 u'user_model_a571vl',
 u'user_model_ad

In [97]:
#df_test_user = df_user[df_user['user_id'].isin(final_test_user_id)]
#df_test_user = df_test_user.drop(['user_id'], axis=1)
df_test_user_2 = df_test_user[user_fea_col].fillna(0)
#df_test_user_2 = df_test_user.fillna(0)
df_test_user_arr = df_test_user_2.values
df_test_user_arr.shape

(2970, 816)

In [98]:
app_category

Index([     u'ART_AND_DESIGN',   u'AUTO_AND_VEHICLES',              u'BEAUTY',
       u'BOOKS_AND_REFERENCE',            u'BUSINESS',              u'COMICS',
             u'COMMUNICATION',              u'DATING',           u'EDUCATION',
             u'ENTERTAINMENT',              u'EVENTS',       u'FAMILY_ACTION',
                   u'FINANCE',      u'FOOD_AND_DRINK',         u'GAME_ACTION',
            u'GAME_ADVENTURE',         u'GAME_ARCADE',          u'GAME_BOARD',
                 u'GAME_CARD',         u'GAME_CASINO',         u'GAME_CASUAL',
          u'GAME_EDUCATIONAL',          u'GAME_MUSIC',         u'GAME_PUZZLE',
               u'GAME_RACING',   u'GAME_ROLE_PLAYING',     u'GAME_SIMULATION',
               u'GAME_SPORTS',       u'GAME_STRATEGY',         u'GAME_TRIVIA',
                 u'GAME_WORD',  u'HEALTH_AND_FITNESS',      u'HOUSE_AND_HOME',
        u'LIBRARIES_AND_DEMO',           u'LIFESTYLE', u'MAPS_AND_NAVIGATION',
           u'MEDIA_AND_VIDEO',             u'MEDICAL

In [99]:
df_test_app0 = df_app2[df_app2['user_id'].isin(test_user_id)]
df_test_app_2 = df_test_user_0[['user_id']].merge(df_test_app0, how = 'left', on = 'user_id')[app_category]
df_test_app_2.head()

,ART_AND_DESIGN,AUTO_AND_VEHICLES,BEAUTY,BOOKS_AND_REFERENCE,BUSINESS,COMICS,COMMUNICATION,DATING,EDUCATION,ENTERTAINMENT,...,PHOTOGRAPHY,PRODUCTIVITY,SHOPPING,SOCIAL,SPORTS,TOOLS,TRANSPORTATION,TRAVEL_AND_LOCAL,VIDEO_PLAYERS,WEATHER
0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,2.0,...,0.0,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0
1,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,2.0,...,0.0,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0
2,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,2.0,...,0.0,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,2.0,1.0,0.0,1.0,0.0,3.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,2.0,1.0,0.0,1.0,0.0,3.0,0.0,1.0,0.0,0.0


In [100]:
#Question: test user without app category
df_test_app_2 = df_test_app_2.fillna(0.01)
df_test_app_arr = df_test_app_2.values
df_test_app_arr.shape

(2970, 53)

In [101]:
# may need to do sampling on df_ua_map
###sophia 20171201
#df_ua_map = df_ua_map.sample(frac=0.3, replace=False)

In [102]:
len(df_ua_map)

139811

In [103]:
#df_train_user = df_train_user_0.merge(df_user_cat, how = 'left', on = 'user_id')

In [104]:
df_final_train_user_article = df_ua_map[(df_ua_map['user_id'].isin(train_user_id))]
df_final_train_user_article = df_final_train_user_article.merge(df_train_user[test_col], how = 'left', on = 'user_id')

In [105]:
print (df_final_train_user_article.shape)
df_final_train_user_article.head()

(139811, 864)


,user_id,article_contentid,user_age_0.0,user_age_1.0,user_age_2.0,user_age_3.0,user_age_4.0,user_gender_0.0,user_gender_1.0,user_gender_2.0,...,user_l2_cat1001040,user_l2_cat1001049,user_l2_cat1001060,user_l2_cat1001091,user_l2_cat1001103,user_l2_cat1001110,user_l2_cat1001117,user_l2_cat2000000,user_l2_cat6000007,user_l2_cat8000001
0,614E304DE3B07EBCDD4E11382578EF5A,Ly7407d4cuC_us,0,0,1,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,614E304DE3B07EBCDD4E11382578EF5A,AY628126a1w_us,0,0,1,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,614E304DE3B07EBCDD4E11382578EF5A,jv70536a3bt_us,0,0,1,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,614E304DE3B07EBCDD4E11382578EF5A,1V70d401fdc_us,0,0,1,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,CD2F2B61135DB27BA3230F91466ED793,jQ6c581bbr2_us,0,0,0,1,0,0,0,1,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [106]:
df_final_train_user_article = df_final_train_user_article.merge(df_tr_article_2.drop(['user_id'],axis = 1), how = 'left', on = 'article_contentid')
df_final_train_user_article.head()

,user_id,article_contentid,user_age_0.0,user_age_1.0,user_age_2.0,user_age_3.0,user_age_4.0,user_gender_0.0,user_gender_1.0,user_gender_2.0,...,l2_cat_1001040,l2_cat_1001049,l2_cat_1001060,l2_cat_1001091,l2_cat_1001103,l2_cat_1001110,l2_cat_1001117,l2_cat_2000000,l2_cat_6000007,l2_cat_8000001
0,614E304DE3B07EBCDD4E11382578EF5A,Ly7407d4cuC_us,0,0,1,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,614E304DE3B07EBCDD4E11382578EF5A,AY628126a1w_us,0,0,1,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,614E304DE3B07EBCDD4E11382578EF5A,jv70536a3bt_us,0,0,1,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,614E304DE3B07EBCDD4E11382578EF5A,1V70d401fdc_us,0,0,1,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,CD2F2B61135DB27BA3230F91466ED793,jQ6c581bbr2_us,0,0,0,1,0,0,0,1,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [107]:
df_final_train_user_article.shape

(139811, 1051)

In [108]:
#df_old_user_article = df_ua_map[df_ua_map['user_id'].isin(old_user_id)]
#df_old_user_article = df_old_user_article.merge(df_train_user[test_col], how = 'left', on = 'user_id')
#df_old_user_article = df_old_user_article.drop_duplicates(subset=['user_id', 'article_contentid'], keep='first').reset_index(drop=True)
#df_old_user_article = df_old_user_article.sort_values('user_id').reset_index(drop=True)
#df_old_user_article = df_old_user_article.merge(df_article.drop(['user_id'],axis = 1), how = 'left', on = 'article_contentid')
#df_old_user_article.head()

In [109]:
#df_final_train = df_final_train_user_article.merge(df_user, how='left', on='user_id')
df_final_train = df_final_train_user_article.fillna(0.1)
#sophia remark: add app data
#df_final_train = df_final_train.merge(df_app2, how='left', on='user_id')

In [110]:
df_final_train.shape

(139811, 1051)

In [111]:
df_final_train = df_final_train.merge(df_app2, how='left', on='user_id')
###sophia remark, check wether there is NAN or all 0 here
#df_final_train = df_final_train.dropna()
#df_final_train.head()

In [112]:
df_final_train.shape

(139811, 1104)

In [113]:
user_feature = [i for i in df_final_train.columns if 'user_' in i and i != 'user_id']
user_feature

['user_age_0.0',
 'user_age_1.0',
 'user_age_2.0',
 'user_age_3.0',
 'user_age_4.0',
 'user_gender_0.0',
 'user_gender_1.0',
 'user_gender_2.0',
 u'user_brand_acer',
 u'user_brand_asus',
 u'user_brand_att',
 u'user_brand_blackberry',
 u'user_brand_blu',
 u'user_brand_cingular_us',
 u'user_brand_coolpad',
 u'user_brand_cubot',
 u'user_brand_dragon touch',
 u'user_brand_emerson',
 u'user_brand_gionee',
 u'user_brand_google',
 u'user_brand_hisense',
 u'user_brand_honor',
 u'user_brand_htc',
 u'user_brand_huawei',
 u'user_brand_intex',
 u'user_brand_kyocera',
 u'user_brand_lenovo',
 u'user_brand_lge',
 u'user_brand_metropcs',
 u'user_brand_micromax',
 u'user_brand_motorola',
 u'user_brand_myphone',
 u'user_brand_nextbit',
 u'user_brand_nextbook',
 u'user_brand_oneplus',
 u'user_brand_rca',
 u'user_brand_samsung',
 u'user_brand_sharp',
 u'user_brand_sky devices',
 u'user_brand_sony',
 u'user_brand_speed',
 u'user_brand_tcl',
 u'user_brand_tcl_metropcs',
 u'user_brand_tct',
 u'user_brand_tct

In [114]:
article_feature = [i for i in df_final_train.columns if 'user_' not in i and i not in ['article_contentid',
 'article_title',
 'article_publisher',
 'l1_cat',
 'l2_cat',
 'l1_cat_w',
 'l2_cat_w',
 'l1_cat_w_x',
 'l2_cat_w_x',
 'l1_cat_w_y',
 'l2_cat_w_y',
 'totalCat'] and i not in app_category]
#article_feature = 
article_feature

[u'dt',
 'l1_cat_0',
 u'l1_cat_0010000000',
 u'l1_cat_1000001',
 u'l1_cat_1000031',
 u'l1_cat_1000044',
 u'l1_cat_1000111',
 u'l1_cat_1000123',
 u'l1_cat_1000267',
 u'l1_cat_1000288',
 u'l1_cat_1000298',
 u'l1_cat_1000374',
 u'l1_cat_1000395',
 u'l1_cat_1000560',
 u'l1_cat_1000620',
 u'l1_cat_1000637',
 u'l1_cat_1000661',
 u'l1_cat_1000721',
 u'l1_cat_1000742',
 u'l1_cat_1000780',
 u'l1_cat_1000849',
 u'l1_cat_1000931',
 u'l1_cat_1000992',
 u'l1_cat_1001039',
 u'l1_cat_1001091',
 u'l1_cat_2000000',
 u'l1_cat_6000007',
 u'l1_cat_8000001',
 'l2_cat_0',
 u'l2_cat_0010000000',
 u'l2_cat_1000001',
 u'l2_cat_1000002',
 u'l2_cat_1000007',
 u'l2_cat_10000101',
 u'l2_cat_1000012',
 u'l2_cat_10000162',
 u'l2_cat_1000017',
 u'l2_cat_10000177',
 u'l2_cat_10000179',
 u'l2_cat_1000025',
 u'l2_cat_1000028',
 u'l2_cat_1000031',
 u'l2_cat_1000032',
 u'l2_cat_1000042',
 u'l2_cat_1000044',
 u'l2_cat_1000050',
 u'l2_cat_10000667',
 u'l2_cat_1000069',
 u'l2_cat_1000072',
 u'l2_cat_1000073',
 u'l2_cat_10000

In [115]:
df_train_user = df_final_train[user_fea_col]
df_train_user.head()

,user_age_0.0,user_age_1.0,user_age_2.0,user_age_3.0,user_age_4.0,user_gender_0.0,user_gender_1.0,user_gender_2.0,user_model_0paj5,user_model_0pcv1,...,user_l2_cat1001040,user_l2_cat1001049,user_l2_cat1001060,user_l2_cat1001091,user_l2_cat1001103,user_l2_cat1001110,user_l2_cat1001117,user_l2_cat2000000,user_l2_cat6000007,user_l2_cat8000001
0,0,0,1,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,1,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,1,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,1,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,0,1,0,0,0,1,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [116]:
df_train_app = df_final_train[app_category]
df_train_app.head()

,ART_AND_DESIGN,AUTO_AND_VEHICLES,BEAUTY,BOOKS_AND_REFERENCE,BUSINESS,COMICS,COMMUNICATION,DATING,EDUCATION,ENTERTAINMENT,...,PHOTOGRAPHY,PRODUCTIVITY,SHOPPING,SOCIAL,SPORTS,TOOLS,TRANSPORTATION,TRAVEL_AND_LOCAL,VIDEO_PLAYERS,WEATHER
0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,2.0,...,0.0,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0
1,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,2.0,...,0.0,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0
2,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,2.0,...,0.0,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0
3,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,2.0,...,0.0,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0
4,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,2.0,...,0.0,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0


In [117]:
df_train_article = df_final_train[article_feature]
df_train_article.head()

,dt,l1_cat_0,l1_cat_0010000000,l1_cat_1000001,l1_cat_1000031,l1_cat_1000044,l1_cat_1000111,l1_cat_1000123,l1_cat_1000267,l1_cat_1000288,...,l2_cat_1001040,l2_cat_1001049,l2_cat_1001060,l2_cat_1001091,l2_cat_1001103,l2_cat_1001110,l2_cat_1001117,l2_cat_2000000,l2_cat_6000007,l2_cat_8000001
0,0.388889,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.222222,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.974894,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.166667,0.0,0.0,0.0,0.439600,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.222222,0.0,0.0,0.0,0.804383,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.611111,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [119]:
#[i for i in df_test_article.columns if i not in df_train_article]

In [120]:
df_train_user.shape

(139811, 816)

In [121]:
df_train_app.shape

(139811, 53)

In [122]:
df_train_article.shape

(139811, 187)

In [123]:
df_test_article = df_tst_article_2[article_feature]
df_test_article.shape

(8257, 187)

In [124]:
len(article_feature)

187

In [125]:
article_feature.append(u'article_contentid')
len(article_feature)

188

In [126]:
df_test_article0 = df_tst_article_2.drop_duplicates(subset=['article_contentid'], keep='first')#[article_feature]
df_test_article0.head()

,article_contentid,dt,l1_cat_0,l1_cat_0010000000,l1_cat_1000001,l1_cat_1000031,l1_cat_1000044,l1_cat_1000111,l1_cat_1000123,l1_cat_1000267,...,l2_cat_1001040,l2_cat_1001049,l2_cat_1001060,l2_cat_1001091,l2_cat_1001103,l2_cat_1001110,l2_cat_1001117,l2_cat_2000000,l2_cat_6000007,l2_cat_8000001
0,AI7da9ea5ts_us,1.000000,0,0.0,0,0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8v7f3eb216n_us,1.000000,0,0.0,0,0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,367e61137BH_us,1.000000,0,0.0,0,0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,Kx81111cfRK_us,1.000000,0,0.0,0,0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Qx784e225N3_us,0.666667,0,0.0,0,0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [127]:
df_test_article_arr1 = df_test_article.values

In [128]:
df_test_article_arr1.shape

(8257, 187)

In [129]:
#df_test_user1 = df_user[df_user['user_id']== 'F4CCDDAA1BAE16B0AF3BCB82801A3CF7']

In [130]:
#df_test_user1.head()

In [131]:
#df_test_user = df_user[df_user['user_id'] == 'F4CCDDAA1BAE16B0AF3BCB82801A3CF7'][user_feature]
#user_array = [df_test_user.values[0] for i in range(0, len(df_test_article))]
#df_test_user = np.asarray(user_array)
#df_test_user.shape

In [132]:
#df_test_app = df_app2[df_app2['user_id']=='F4CCDDAA1BAE16B0AF3BCB82801A3CF7'][app_category]
#app_array = [df_test_app.values[0] for i in range(0, len(df_test_article))]
#df_test_app = np.asarray(app_array)
#df_test_app.shape

In [133]:
#final_test_user_id

In [134]:
y_train = [1 for i in range(len(df_train_user.values))]
len(y_train)

139811

### Build Model By Keras (User Vs App Vs Article)

In [135]:
df_train_user.columns

Index([      u'user_age_0.0',       u'user_age_1.0',       u'user_age_2.0',
             u'user_age_3.0',       u'user_age_4.0',    u'user_gender_0.0',
          u'user_gender_1.0',    u'user_gender_2.0',   u'user_model_0paj5',
         u'user_model_0pcv1',
       ...
       u'user_l2_cat1001040', u'user_l2_cat1001049', u'user_l2_cat1001060',
       u'user_l2_cat1001091', u'user_l2_cat1001103', u'user_l2_cat1001110',
       u'user_l2_cat1001117', u'user_l2_cat2000000', u'user_l2_cat6000007',
       u'user_l2_cat8000001'],
      dtype='object', length=816)

In [136]:
df_train_user.shape, df_train_app.shape

((139811, 816), (139811, 53))

In [137]:
df_train_user_arr = df_train_user[user_fea_col].values
df_train_app_arr = df_train_app.values

In [138]:
df_train_user_arr.shape, df_train_app_arr.shape

((139811, 816), (139811, 53))

In [139]:
df_train_article.shape

(139811, 187)

In [140]:
df_train_article_arr = df_train_article.values
df_train_article_arr.shape

(139811, 187)

In [141]:
y_train = [1 for i in range(len(df_train_user))]
len(y_train)

139811

In [142]:
from keras.models import Sequential, Model
from keras.layers import Dense, Merge, Activation, Dropout, Reshape

Using TensorFlow backend.


In [143]:
# create user model
userModel1 = Sequential()
userModel1.add(Dense(500, input_shape=(816,)))
userModel1.add(Activation('tanh'))
userModel1.add(Dropout(0.4))
userModel1.add(Dense(50, input_shape=(500,)))
userModel1.add(Activation('tanh'))
userModel1.add(Dropout(0.4))
userModel1.add(Dense(30, input_shape=(50,)))
userModel1.add(Activation('tanh'))
userModel1.add(Dropout(0.4))
userModel1.add(Dense(10, input_shape=(30,)))
userModel1.add(Activation('tanh'))
userModel1.add(Reshape((10,1)))

In [145]:
# create article model
articleModel = Sequential()
articleModel.add(Dense(50, input_shape=(187,)))
articleModel.add(Activation('tanh'))
articleModel.add(Dropout(0.4))
articleModel.add(Dense(30, input_shape=(50,)))
articleModel.add(Activation('tanh'))
articleModel.add(Dropout(0.4))
articleModel.add(Dense(10, input_shape=(30,)))
articleModel.add(Activation('tanh'))
articleModel.add(Reshape((10,1)))

In [146]:
model1 = Sequential()
model1.add(Merge([userModel1,articleModel], mode='cos', dot_axes=1, output_shape=(1,)))
model1.add(Reshape((1,)))

In [147]:
# create user model 2
userModel2 = Sequential()
userModel2.add(Dense(30, input_shape=(53,)))
userModel2.add(Activation('tanh'))
userModel2.add(Dropout(0.4))
userModel2.add(Dense(10, input_shape=(30,)))
userModel2.add(Activation('tanh'))
userModel2.add(Reshape((10,1)))

In [148]:
model2 = Sequential()
model2.add(Merge([userModel2,articleModel], mode='cos', dot_axes=1, output_shape=(1,)))
model2.add(Reshape((1,)))

In [149]:
model=Sequential()
model.add(Merge([model1,model2],mode='concat'))
model.add(Dense(1, input_shape=(2,)))
model.add(Reshape((1,)))
#model.add(Activation('normalization'))
#model.add(Reshape(1,))
#model.add(Activation('softmax'))

In [150]:
# Compile model
model.compile(loss='cosine_proximity', optimizer='adam', metrics=['accuracy'])

In [151]:
len(df_test_article_arr1)

8257

In [152]:
model.fit([df_train_user_arr ,df_train_article_arr,df_train_app_arr] ,y_train, nb_epoch=1, batch_size=32)

Epoch 1/1
139811/139811 [==============================] - 53s - loss: -0.9806 - acc: 0.6891    


In [ ]:
# Compile model 1
#model1.compile(loss='cosine_proximity', optimizer='adam', metrics=['accuracy','precision','recall'])
#model1.fit([df_train_user_arr ,df_train_article_arr] ,y_train, nb_epoch=1, batch_size=32)

# Compile model 2
#model2.compile(loss='cosine_proximity', optimizer='adam', metrics=['accuracy','precision','recall'])
#model2.fit([df_train_app_arr ,df_train_article_arr] ,y_train, nb_epoch=1, batch_size=32)

In [ ]:
#y_pred = model.predict([df_test_user,df_test_article_arr1, df_test_app], batch_size=32, verbose=0)
#y_pred_1 = np.hstack(y_pred)
#np.argsort(y_pred_1)[:100]

### Calculate precision and recall

In [153]:
df_test_user_arr1 = df_test_user[user_fea_col].values
df_test_user_arr1.shape

(2970, 816)

In [154]:
#sophia remark get the mapping of user article
#df_ua_map = df_trData[['user_id', 'article_contentid']]

#Sophia remark: get test user list
#df_test_user0 = df_user[df_user['user_id'].isin(final_test_user_id)]
#df_test_user = df_user[df_user['user_id'].isin(final_test_user_id)]
#df_test_user = df_test_user.drop(['user_id'], axis=1)
#df_test_user = df_test_user.fillna(0)
df_test_user_arr1 = df_test_user[user_fea_col].values
#df_test_user_arr1.shape

#Sophia remark: get test app list
df_test_app0 = df_app2[df_app2['user_id'].isin(test_user_id)]
df_test_app = df_user[df_user['user_id'].isin(test_user_id)].merge(df_test_app0, how = 'left', on = 'user_id')[app_category]
df_test_app = df_test_app.fillna(0)
df_test_app_arr1 = df_test_app.values
#df_test_app_arr1.shape

#sophia remark: get article list
#df_test_article0 = df_tst_article.drop_duplicates(subset=['article_contentid'], keep='first')#[article_feature]
#df_test_article0.head()
col = df_train_article.columns
df_test_article_arr1 = df_test_article[col].values

In [155]:
df_test_article[col].shape

(8257, 187)

In [156]:
print df_test_article[col].shape
df_test_article[col].head()

(8257, 187)


,dt,l1_cat_0,l1_cat_0010000000,l1_cat_1000001,l1_cat_1000031,l1_cat_1000044,l1_cat_1000111,l1_cat_1000123,l1_cat_1000267,l1_cat_1000288,...,l2_cat_1001040,l2_cat_1001049,l2_cat_1001060,l2_cat_1001091,l2_cat_1001103,l2_cat_1001110,l2_cat_1001117,l2_cat_2000000,l2_cat_6000007,l2_cat_8000001
0,1.000000,0,0.0,0,0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1.000000,0,0.0,0,0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.000000,0,0.0,0,0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1.000000,0,0.0,0,0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.666667,0,0.0,0,0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [157]:
df_train_article.head()

,dt,l1_cat_0,l1_cat_0010000000,l1_cat_1000001,l1_cat_1000031,l1_cat_1000044,l1_cat_1000111,l1_cat_1000123,l1_cat_1000267,l1_cat_1000288,...,l2_cat_1001040,l2_cat_1001049,l2_cat_1001060,l2_cat_1001091,l2_cat_1001103,l2_cat_1001110,l2_cat_1001117,l2_cat_2000000,l2_cat_6000007,l2_cat_8000001
0,0.388889,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.222222,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.974894,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.166667,0.0,0.0,0.0,0.439600,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.222222,0.0,0.0,0.0,0.804383,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.611111,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [158]:
df_test_article_arr1.shape

(8257, 187)

In [159]:
df_test_app_arr1.shape

(2970, 53)

In [160]:
t_u = df_test_user[user_fea_col]
t_u = t_u.reset_index(drop=True)
t_u.head()

,user_age_0.0,user_age_1.0,user_age_2.0,user_age_3.0,user_age_4.0,user_gender_0.0,user_gender_1.0,user_gender_2.0,user_model_0paj5,user_model_0pcv1,...,user_l2_cat1001040,user_l2_cat1001049,user_l2_cat1001060,user_l2_cat1001091,user_l2_cat1001103,user_l2_cat1001110,user_l2_cat1001117,user_l2_cat2000000,user_l2_cat6000007,user_l2_cat8000001
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [161]:
df_test_user0 = df_tst_user

In [162]:
#df_test_article_arr1[0:5]

In [163]:
l = len(df_test_article_arr1)
uid = df_test_user0['user_id'].tolist()[0]
user = np.array([df_test_user_arr1[0]]*l)
user
app = np.array([df_test_app_arr1[0]]*l)
#apply user and article array to do predict
up = pd.DataFrame(model.predict([user,df_test_article_arr1,app], batch_size=32))
df_test_article1=df_test_article0.reset_index(drop = True)\
                            .merge(up, how = 'left', left_index=True, right_index=True)\
                            .rename(columns={0:'u0_score'})[['article_contentid', 'u0_score']]
df_test_article1 = df_test_article1.sort_values(by='u0_score', ascending=False)
df_test_article1['y_pred'] = 0
df_test_article1[:5]['y_pred']=1
top5_art = df_test_article1[:5]['article_contentid'].values


top5_art

/Users/sophia.gao/.pyenv/versions/miniconda2-latest/lib/python2.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


array([u'MP2cbde13wV_us', u'SB665e92936_us', u'9y6bc6224vB_us',
       u'3V6fded5eYe_us', u'xz643f403q7_us'], dtype=object)

In [167]:
l = len(df_test_article_arr1)
dic = {}
for idx, user in enumerate(df_test_user_arr1):
    #for each user-article pair: get the user id and user info
    uid = df_test_user0['user_id'].tolist()[idx]
    user = np.array([df_test_user_arr1[idx]]*l)
    app = np.array([df_test_app_arr1[idx]]*l)
    #apply user and article array to do predict
    up = pd.DataFrame(model.predict([user,df_test_article_arr1, app], batch_size=32))
    df_test_article1=df_test_article0.reset_index(drop = True)\
                            .merge(up, how = 'left', left_index=True, right_index=True)\
                            .rename(columns={0:'u0_score'})[['article_contentid', 'u0_score']]
    df_test_article1 = df_test_article1.drop_duplicates(['article_contentid'])
    df_test_article1 = df_test_article1.sort_values(by='u0_score', ascending=False)
    #df_test_article1['y_pred'] = 0
    dic[uid] = df_test_article1[:5]['article_contentid']
    print uid, ":", dic[uid].tolist()


CD2F2B61135DB27BA3230F91466ED793 : [u'MP2cbde13wV_us', u'SB665e92936_us', u'9y6bc6224vB_us', u'3V6fded5eYe_us', u'xz643f403q7_us']
5BA72C11B1E94BA50F1E66F4B1362CE6 : [u'717d9bbfaru_us', u'0h7e77e5dmL_us', u'2G7a282325k_us', u'7l7fbad25Fj_us', u'iu7fc3c93kd_us']
40BE601804C6B2E842FFC30F0EDBAC23 : [u'5h67712c82w_us', u'aI77d391fch_us', u'Fv7d13937nQ_us', u'ga8105583Yt_us', u'rS7e24cbb7t_us']
E1295F6622EA8DA36DE678772D9ED152 : [u'2e7fda398Jw_us', u'Nz7fffa9fp1_us', u'hj80a0439fM_us', u'ay7fcedf977_us', u'0r7dc6e4aez_us']
F9B30E5BDD8009EBA52C1F5518D9E332 : [u'2e7fda398Jw_us', u'Vu5e0467ep2_us', u'dA7f7be30j7_us', u'Gz796c118nv_us', u'Na7a19938Nd_us']
FDA822F54256CAE5209A7FD11B87503F : [u'2e7fda398Jw_us', u'Hf7e5f826VB_us', u'0r7dc6e4aez_us', u'j47c8b31asv_us', u'0G7fd47460d_us']
34FFE2FE9783A2404A5BE490D771CEB5 : [u'sZ5da6036Ex_us', u'S581176c1aQ_us', u'DE8006b5eXA_us', u'pT814b960R7_us', u'LW7fbb1e3TY_us']
6570B22D8B503392AC07E9AE7F130F29 : [u'fu79ccfd26K_us', u'j47f00464bW_us', u'In7ef4d

3A98508639A6C25753B7EBF925D0EEDC : [u'hj80a0439fM_us', u'Nz7fffa9fp1_us', u'ay7fcedf977_us', u'dA7f7be30j7_us', u'Zn5cc21db7C_us']
9D9B416A60685E208CE23216A1795FF8 : [u'mw5dfa494oC_us', u'iX69d4ecbI3_us', u'Zn5cc21db7C_us', u'Mo77f2f4d8Y_us', u'BP726683dAd_us']
BFE8C029CAC842F495A0DF7F9915F770 : [u'Nz7fffa9fp1_us', u'hj80a0439fM_us', u'ay7fcedf977_us', u'Zn5cc21db7C_us', u'mw5dfa494oC_us']
C215516D2B3A361E5894DF1895058491 : [u'Mo77f2f4d8Y_us', u'pD78433edAS_us', u'Sr73f3cd8AY_us', u'OC810c8f8m1_us', u'mw5dfa494oC_us']
095E4898A396707D75E4BC2E454EF183 : [u'Zn5cc21db7C_us', u'mw5dfa494oC_us', u'ay7fcedf977_us', u'Nz7fffa9fp1_us', u'hj80a0439fM_us']
A373AE178CC0F261AB3ADF69A58CB1B7 : [u'g680031b5w7_us', u'Pu629e28brD_us', u'Wk80f662daH_us', u'sz7575d96wK_us', u'H98153751zP_us']
58721A3247F0CD9132BC0CD7B1EED285 : [u'kn7e811f5Np_us', u'tU8050ef8gA_us', u'TX7fcdf57Bg_us', u'mK7f3f366aY_us', u'nD805de88XP_us']
6BD8ACAB2B18B2D1768FF94C7C7EA180 : [u'Pe619e986YW_us', u'Zh68bb9782u_us', u'Jq72e59

0C4E08F07331B5EEAD200B7BB01BC0AD : [u'3d7e3e08cGd_us', u'SB70d37247x_us', u'8x7fc8516S2_us', u'id71ec75eCs_us', u'hH765fccaXK_us']
C3611406D184B49C60B43E9ED37BF1B2 : [u'P27edf2743N_us', u'qO8065529QO_us', u'He80216e2Fg_us', u'Id663c1e0py_us', u'1U8152055Qv_us']
F659C833AF90021EDCE3ABB65B52A137 : [u'Oj70b244ekd_us', u'Ch74fb215yG_us', u'hn752da731A_us', u'Zh68bb9782u_us', u'Jq72e59e8Hr_us']
5BF88795F8FCD945913844BE232BAECE : [u'RE72b6f36ZK_us', u'OY6a0cce1Xj_us', u'yk6e3b7e4aI_us', u'fo7fb8d5fao_us', u'az7e3757eDr_us']
B2D75887E965F416B4B8BC5A48CF21AF : [u'0h7e77e5dmL_us', u'2G7a282325k_us', u'7l7fbad25Fj_us', u'iu7fc3c93kd_us', u'717d9bbfaru_us']
60BB8A4ECE043A274ACB8CDF271901C5 : [u'0h7e77e5dmL_us', u'2G7a282325k_us', u'7l7fbad25Fj_us', u'iu7fc3c93kd_us', u'Ne7f42ebaRE_us']
F4943A2381892770411E2958AC94436D : [u'Xt6bd226fdo_us', u't166a8b7b2X_us', u'wR77515800R_us', u'GS6e4c9b6W8_us', u'bI7c5a091Wc_us']
72075DA1DB250C972FAED076BA25F868 : [u'wR77515800R_us', u't166a8b7b2X_us', u'Xt6bd22

BA4498DB51B4DBAE457C118DC9DE3380 : [u'tP7f34d60La_us', u'sW80c9f07v8_us', u'Ad767e6farp_us', u'3D6332b51fr_us', u'w280edcb5DD_us']
5500AA1AB2B81D2DB74BE46ABD28133D : [u'wR77515800R_us', u't166a8b7b2X_us', u'Xt6bd226fdo_us', u'GS6e4c9b6W8_us', u'bI7c5a091Wc_us']
286115A9EB489602DA4C05CC6B6987FB : [u'Ad767e6farp_us', u'rl7d33258N9_us', u'Z180492d0wi_us', u'097f90c94ge_us', u'dB5dc5ef4C2_us']
877FDFB3591052E1B34ADEF7BE50D5AF : [u'Ad767e6farp_us', u'w280edcb5DD_us', u'AT610c73dLV_us', u'dB5dc5ef4C2_us', u'Vl708eb3bDJ_us']
33C1723C9BC88AEE3A2F2879537AD01C : [u'w280edcb5DD_us', u'AT610c73dLV_us', u'dB5dc5ef4C2_us', u'0r7dc6e4aez_us', u'rl7d33258N9_us']
54FF1517924158E842967A675207C52C : [u'0r7dc6e4aez_us', u'b4655460fvY_us', u'ws72432a5na_us', u'KB62eb7dfp1_us', u'w280edcb5DD_us']
ABC793EC14A1DBAFECB101CC2F2C0E47 : [u'w280edcb5DD_us', u'AT610c73dLV_us', u'dB5dc5ef4C2_us', u'Vl708eb3bDJ_us', u'0r7dc6e4aez_us']
931C2EB24AC6F94639308CB4ED5781C0 : [u'wR77515800R_us', u't166a8b7b2X_us', u'Xt6bd22

E946553662F1A96BA2B6CEB322022D84 : [u'0h7e77e5dmL_us', u'2G7a282325k_us', u'7l7fbad25Fj_us', u'J46d06498kN_us', u'iu7fc3c93kd_us']
0BF688286703A0A2405F937ADB4BFF45 : [u'0r7dc6e4aez_us', u'q97c98b65Qh_us', u'217fa22feMe_us', u'fp7ffb1a3C0_us', u'4680fedf5BV_us']
261FED70BFED3D8B9C5838C45AEBBE12 : [u'Ge7b17d0enh_us', u'1I7ef8256tK_us', u'tz7e0951e4C_us', u'SG7b2a192go_us', u'Rj80673edOD_us']
72045B6A53B39C544BC2260E36FAD90C : [u'In7ef4d72p6_us', u'DX815d838e8_us', u'1U8152055Qv_us', u'fu79ccfd26K_us', u'Nu79ef27774_us']
A22AC558962FC420939B0D97E495C5DC : [u'oE805f43ar3_us', u'Up7ed2f900f_us', u'Eb8090554iM_us', u'rI7e1564b6F_us', u'wq7f0ad03TS_us']
EA24D50F1BE49C0724BE6B1A7E49FFE9 : [u'Pe619e986YW_us', u'Zh68bb9782u_us', u'Jq72e59e8Hr_us', u'07690c4b865_us', u'UB6a6c55eFK_us']
A149D2390DB6559C3CF9D5C2F3F4E425 : [u'1U8152055Qv_us', u'fu79ccfd26K_us', u'In7ef4d72p6_us', u'DX815d838e8_us', u'Nu79ef27774_us']
2A0EAFFFE34930BFD66C65FA890749F7 : [u'9y6bc6224vB_us', u'xz643f403q7_us', u'g0613c5

...

C99D84988C6C69ED695FAA3CE60140FB : [u'V080ea49cFh_us', u'FJ7f10d4deH_us', u'tX7e5b5bcXL_us', u'tp7ed0d65m8_us', u'Cj7e8bd55fx_us']
168A8A8983E7FEE2C7977D96BFF52C69 : [u'tX7e5b5bcXL_us', u'Cj7e8bd55fx_us', u'cT81417db3j_us', u'i77ba854cxy_us', u'pv7e20020h5_us']
C4E41C1AA8080266F1036AB7C4F92431 : [u'f87f2220b7W_us', u'LM7e2873cOT_us', u'us80398e3zE_us', u'9u80238cef0_us', u'Cy7f76744qc_us']
204C88A91FE3AFD081008F1722C72C6F : [u'V080ea49cFh_us', u'BX7fb17f23Q_us', u'i87f8d14c2M_us', u'Mt7df2b26u9_us', u'Mk157aeedQU_us']
7B410DE352E75238760AF606F4791476 : [u'kn7e811f5Np_us', u'tU8050ef8gA_us', u'p47c91e74zL_us', u'Wk80f662daH_us', u'sz7575d96wK_us']
2BC3DC049B7C8D6508D922C03F4D6672 : [u'Un7f718ef2o_us', u'Ol80a2d03X1_us', u'nD805de88XP_us', u'mK7f3f366aY_us', u'TX7fcdf57Bg_us']
E8E98315B6B6867E4DA32747F919BD41 : [u'9r776732egM_us', u'5x63cc2019p_us', u'Oj70b244ekd_us', u'Ch74fb215yG_us', u'hn752da731A_us']
F964B1A607E9284396DC942910E997AE : [u'Oj70b244ekd_us', u'Ch74fb215yG_us', u'hn752da

AA65423CD1399C42B6DD1EC8C4D98FAE : [u'vw807fc55Fv_us', u'He80216e2Fg_us', u'qO8065529QO_us', u'P27edf2743N_us', u'Sn806a6d6O2_us']
172C918555B875AB3596679E01FD7CFF : [u'Fv7d13937nQ_us', u'ga8105583Yt_us', u'ir7d6ff10fj_us', u'MN812b821um_us', u'rS7e24cbb7t_us']
DFC22F78B21768C3EC3CEA2AA09F2734 : [u'1U8152055Qv_us', u'In7ef4d72p6_us', u'DX815d838e8_us', u'fu79ccfd26K_us', u'Nu79ef27774_us']
BD90B2DC38D2F45C02A9D26C5714D052 : [u'vw807fc55Fv_us', u'P27edf2743N_us', u'qO8065529QO_us', u'He80216e2Fg_us', u'ou816d8ef1V_us']
31160751F648ED63C4981342604F77B5 : [u'Ey73325a4PK_us', u'3Q66f4a401h_us', u'jx6217d8cPX_us', u'sM653c2c6jI_us', u'Gz796c118nv_us']
231C383085903EC39287A769F16A7E2C : [u'Nj6e691c0hZ_us', u'kE70a96adWc_us', u'Fh653aea4h3_us', u'sM653c2c6jI_us', u'3Q66f4a401h_us']
01D0068F5B8740BC54D2A832903E3542 : [u'nQ713910fsk_us', u'PE7df9addHr_us', u'sJ7d382fdrD_us', u'SD8058f9fI3_us', u'VF7f39ca1XY_us']
00E1ED486DCC8EC45179B6A0D705A7AA : [u'jx6217d8cPX_us', u'5H73f101fQ0_us', u'sM653c2

ADD606D2F1ED192F9412FC9D8C802178 : [u'Oj70b244ekd_us', u'Ch74fb215yG_us', u'hn752da731A_us', u'IX7152777C1_us', u'5x63cc2019p_us']
D2F11205A0CC78F5FF675CC8011E51F1 : [u'wI75cbf1bWi_us', u'qk75ee88cTA_us', u'al7c71911Fe_us', u'eR7f459760N_us', u'2r80dfecdmS_us']
F59D0B96709EEAA597EA4B30E5B58C38 : [u'GS6e4c9b6W8_us', u'SB665e92936_us', u'MP2cbde13wV_us', u'bI7c5a091Wc_us', u'9y6bc6224vB_us']
3DB12A45804E9D5712C990985BFE8670 : [u'uJ6bbe746yR_us', u'4o6c729a4aS_us', u'US7e84931Ol_us', u'4J7e9245e4f_us', u'Ws73dce7aVP_us']
CCF4C0377E80AFB88DEB11F8751785A8 : [u'NR6ee5b8bVZ_us', u'367e61137BH_us', u'VP7fadff4Je_us', u'jO7f12d4ftK_us', u'DA801ebed26_us']
65174C9C227D926843E7329188F91F4A : [u'ou816d8ef1V_us', u'G080173abko_us', u'Jd8134851i1_us', u'vw807fc55Fv_us', u'Qd7f57c4bNr_us']
B7BBC622DA31D335C6AA8344E7A465A4 : [u'OH6199740Az_us', u'UP7ceb224IC_us', u'kx7d1f099sa_us', u'k480e659205_us', u'WJ7f85fdanw_us']
2DBF85D884061CB8B8D481C0D9420C11 : [u'FV57f67a31o_us', u'8e80ef36ca5_us', u'uO7c562

DA8D6D7538049B4311095F3AF3ECF5EC : [u'6a81442f9yY_us', u'8G71adda1n4_us', u'Uo631e4a48x_us', u'Lg7c33d92OE_us', u'9o7a3abc3DD_us']
1E32578843B75B8442DA01CEB59F57C1 : [u'kD5b88e54a9_us', u'PE7df9addHr_us', u'9880740caYR_us', u'VF7f39ca1XY_us', u'SD8058f9fI3_us']
9F68CECB20F53F37640B718BC30EBA1D : [u'nQ713910fsk_us', u'7r6515781uH_us', u'PE7df9addHr_us', u'LW7fbb1e3TY_us', u'pT814b960R7_us']
2EF736586A8D6F3719704D062FFDA253 : [u'UP7ceb224IC_us', u'kx7d1f099sa_us', u'k480e659205_us', u'WJ7f85fdanw_us', u'pI7fb5349eT_us']
7A71DFCF8A8741FE5EE304636BF7640D : [u'ou816d8ef1V_us', u'vw807fc55Fv_us', u'KS7cb045fxy_us', u'He80216e2Fg_us', u'qO8065529QO_us']
F529AEA6FF0B51F222EA962E88184FE4 : [u'Md8097ceeRq_us', u'G9815c73c0V_us', u'g4817d624no_us', u'ox7f34fe5B0_us', u'GS6e4c9b6W8_us']
8D083A874117AB9B61F3FE1810FB68E6 : [u'Pe619e986YW_us', u'07690c4b865_us', u'Zh68bb9782u_us', u'Jq72e59e8Hr_us', u'3V6fded5eYe_us']
6F05192C438BF5E5B2266472BCD9078C : [u'iX69d4ecbI3_us', u'mw5dfa494oC_us', u'Zn5cc21

6E4703D2D567D7FD662A9DEE74810CBA : [u'f87f2220b7W_us', u'us80398e3zE_us', u'LM7e2873cOT_us', u'9u80238cef0_us', u'Cy7f76744qc_us']
996873AE8E3341D7B225013DE1F1F735 : [u'd37fc38fbQY_us', u'6O7f596a551_us', u'f1723651dJj_us', u'rE7f8536csW_us', u'lg7fbc9c9eS_us']
810B6812E946ED488820ACF60EEEA4BA : [u'd37fc38fbQY_us', u'6O7f596a551_us', u'f1723651dJj_us', u'rE7f8536csW_us', u'We7e4612evh_us']
B6D3CBCEB6CDB809FB057C47A27D35FD : [u'0r7dc6e4aez_us', u'ws72432a5na_us', u'Fp64ce79dJe_us', u'Cm75cca06tI_us', u'KB62eb7dfp1_us']
E33545D324718D525F0A4759CD7EABFA : [u'd37fc38fbQY_us', u'6O7f596a551_us', u'We7e4612evh_us', u'f1723651dJj_us', u'rE7f8536csW_us']
6D65E94F235B447BB43D5825CCE449F1 : [u'ou816d8ef1V_us', u'P27edf2743N_us', u'qO8065529QO_us', u'He80216e2Fg_us', u'vw807fc55Fv_us']
BBF08DEB4D876661F4D07D6A90A7DD3E : [u'1U8152055Qv_us', u'In7ef4d72p6_us', u'DX815d838e8_us', u'fu79ccfd26K_us', u'Nu79ef27774_us']
D84C1534B6E3DD10D0843B8D02350E69 : [u'MN812b821um_us', u'vT763cd992h_us', u'qs6946a

In [559]:
l = len(df_test_article_arr1)
precision = list()
recall = list()

for idx, user in enumerate(df_test_user_arr1):
    if idx > 6:
        break;
    #for each user-article pair: get the user id and user info
    uid = df_test_user0['user_id'].tolist()[idx]
    user = np.array([df_test_user_arr1[idx]]*l)
    app = np.array([df_test_app_arr1[idx]]*l)
    #apply user and article array to do predict
    up = pd.DataFrame(model.predict([user,df_test_article_arr1, app], batch_size=32))
    df_test_article1=df_test_article0.reset_index(drop = True)\
                            .merge(up, how = 'left', left_index=True, right_index=True)\
                            .rename(columns={0:'u0_score'})[['article_contentid', 'u0_score']]
    df_test_article1 = df_test_article1.sort_values(by='u0_score', ascending=False)
    df_test_article1['y_pred'] = 0
    df_test_article1[:5]['y_pred']=1
    df_match = df_ua_map[df_ua_map['user_id'] == uid][['article_contentid']]
    df_match['y_true'] = 1
    df_test_article1 = df_test_article1.merge(df_match, how = 'left', on = 'article_contentid')\
                                       .fillna(0)
    tp = df_test_article1[(df_test_article1['y_pred'] == 1) & (df_test_article1['y_true'] == 1)].shape[0]
    fp = df_test_article1[(df_test_article1['y_pred'] == 1) & (df_test_article1['y_true'] == 0)].shape[0]
    fn = df_test_article1[(df_test_article1['y_pred'] == 0) & (df_test_article1['y_true'] == 1)].shape[0]

    p = tp/(float(tp+fp))
    r = tp/(float(tp+fn))
    print (idx, uid, p,r)
    if p != 0:
        precision.append(p)
    if r != 0:
        recall.append(r)
    
print (precision, recall)

/Users/sophia.gao/.pyenv/versions/miniconda2-latest/lib/python2.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(0, u'CD2F2B61135DB27BA3230F91466ED793', 0.0, 0.0)
(1, u'5BA72C11B1E94BA50F1E66F4B1362CE6', 0.0, 0.0)


ZeroDivisionError: float division by zero

In [ ]:
df_test_article1.head()

In [ ]:
df_test_article1[df_test_article1['article_contentid'].isin(list(df_trData[df_trData['user_id'] == 'B3B51127D5FB5B39E4273BE65CACA28C']['article_contentid']))]

In [ ]:
df_ua_map[df_ua_map['user_id'] == '5BA72C11B1E94BA50F1E66F4B1362CE6']

In [ ]:
list(df_trData[df_trData['user_id'] == '5BA72C11B1E94BA50F1E66F4B1362CE6']['article_contentid'])

In [ ]:
t_u.head()

In [ ]:
t_u['index'] = t_u.index
t_u_group = t_u.groupby(by=[i for i in t_u.columns if i != 'index']).agg({'index':'nunique'}).reset_index(drop=True)
t_u_group = t_u_group.sort_values(by=['index'], ascending=False)
t_u_group.head()

# Deal with test data

In [ ]:
# get df_test_user

In [ ]:
# get df_test_app

In [ ]:
# get df_test_article

In [ ]:
'''
for idx, user in enumerate(df_test_user_arr2):
    if idx > 3:
        break;
     #for each user-article pair: get the user id and user info
    uid = df_test_user2['user_id'].tolist()[idx]
    user = np.array([df_test_user_arr2[idx]]*l)
    app = np.array([df_test_app_arr2[idx]]*l)
    
    #apply user and article array to do predict
    up = pd.DataFrame(model.predict([user,df_test_article_arr2, app], batch_size=32, verbose=0))
    df_test_article1=df_test_article0.reset_index(drop = True)\
                            .merge(up, how = 'left', left_index=True, right_index=True)\
                            .rename(columns={0:'u0_score'})[['article_contentid', 'u0_score']]
    df_test_article1 = df_test_article1.sort_values(by='u0_score', ascending=False)
    df_test_article1[:5]
'''